<a href="https://colab.research.google.com/github/tb000330/Research_Assistant/blob/main/Climate%20finance/Bayesian_Optimization(XGB_pannel_df).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import datetime
from tqdm import tqdm

import seaborn as sns
from matplotlib.patches import Patch
from matplotlib import pyplot as plt
plt.rc('font', family='NanumGothic')
plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import TimeSeriesSplit 
from matplotlib.patches import Patch 

In [ ]:
panel = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/panel_df.csv')
panel = panel.iloc[:,1:].reset_index(drop=True)
panel.tail(3)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,코드대분류,코드세분류,코드세세분류,코드소분류,코드중분류,주거래은행,최초상장일자,상장일자,unlist_date,unlist_reason
111901,A950180,SNK,7.0,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,...,J,5821.0,5821.0,582.0,58.0,미츠호은행,20190507.0,20190507.0,2022-05-18,상장폐지 신청('22.04.14)
111902,A950180,SNK,7.0,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,...,J,5821.0,5821.0,582.0,58.0,미츠호은행,20190507.0,20190507.0,2022-05-18,상장폐지 신청('22.04.14)
111903,A950180,SNK,7.0,2021,0.694562,0.101859,-0.592702,2.830798,9.826040,1.322351,...,J,5821.0,5821.0,582.0,58.0,미츠호은행,20190507.0,20190507.0,2022-05-18,상장폐지 신청('22.04.14)


# #1. 전처리 & 파생변수

In [ ]:
panel = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/panel_df.csv')
panel = panel.iloc[:,1:].reset_index(drop=True)

# return 변수 float
panel['return'] = panel['return'].astype(str)
panel['return'] = [re.sub(",", "", row) for row in panel['return']]
panel['return'] = panel['return'].astype(float)

# astype 또는 to_datetime로 변환
# panel['year'] = panel['year'].astype(str)
panel['최초상장일자'] = panel['최초상장일자'].astype(str)
panel['상장일자'] = panel['상장일자'].astype(str)
panel['최초상장일자'] = [re.sub(".0$", "", row) for row in panel['최초상장일자']]
panel['상장일자'] = [re.sub(".0$", "", row) for row in panel['상장일자']]
panel['최초상장일자'] = pd.to_datetime(panel['최초상장일자'])
panel['상장일자'] = pd.to_datetime(panel['상장일자'])

panel['unlist_year'] = panel['unlist_date'].str.slice(stop=4)
# panel['unlist_date'] = pd.to_datetime(panel['unlist_date'])
# panel['Accounting_Closing'] = panel['Accounting_Closing'].astype(int) #오류남
panel['unlist_reason'] = panel['unlist_reason'].astype(str)

# 금융회사 제외
panel = panel[panel['대분류']!='금융 및 보험업']
panel = panel.drop(['대분류', '세분류', '세세분류','소분류', '중분류', '코드세분류', '코드세세분류', '코드소분류', '코드중분류', '주거래은행'], axis=1)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## (2) 상장폐지 사유 전처리 & 카테고리화

In [ ]:
panel['unlist_reason_ctg'] = panel['unlist_reason']

## 상장폐지신청 dict로 class화 
ctg_dict = {}
ctg_list = ['(상장폐지 ?신청)|(^신청)','(신청)|(요청)|(자진)',"(상장$)|(^시장)|(^증권)",'(편입)|(흡수)',"지주회사", "영업의 양도", "기타", "계약이전",
            '해산', "부도", '전액', '자본', "(^감사(으로)? ?의견( 의견)? ?거절)|(범위 ?제한)|(감사의견 ?부적정)|(감사의견거절)", "경영", 
            "(시가 ?총액)|(시총)","주식","보통주", "정리", "영업", "유예", "정지", "제출", "부진", "액면가", "불성실", "화의 불인가", "회생"]

for ctg in ctg_list:
  r = re.compile(ctg) 
  v = list(set([row for row in panel['unlist_reason_ctg'] if r.search(row) is not None]))
  k = min(v, key=len)
  ctg_dict[k] = v
  panel['unlist_reason_ctg'] = panel['unlist_reason_ctg'].replace(v, '0')

# keys 이름 수정
# ctg_dict["자본잠식률 50% 이상"] = ctg_dict["2회연속 자본잠식률 50%이상"]
# ctg_dict["불성실공시법인지정"] = ctg_dict["최근 2년간 3회이상 불성실공시법인으로 지정"]

# del ctg_dict["2회연속 자본잠식률 50%이상"], ctg_dict["최근 2년간 3회이상 불성실공시법인으로 지정"]

In [ ]:
ctg_dict.keys()

dict_keys(['신청에 의한 상장폐지', '자진등록취소', '코스닥시장 상장', '피흡수합병', '지주회사(최대주주등)의 완전자회사화 등', '주된영업의 양도', '기타 등록취소', '계약이전', '해산 사유 발생', '부도', '자본전액잠식', '2연속 자본잠식률 50% 이상', '감사의견거절', '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당', '시가총액 미달', '주식양도 제한', '보통주 주가수준미달', '회사정리절차 폐지 결정', '영업활동정지 6월 계속', '상장폐지유예기간종료', '당좌거래정지', '공시서류 미제출(분기보고서) 후 차기보고서 미제출', '2분기 연속 거래실적부진사유 미해소', '액면가액일정비율 미달', '최근 2년간 3회이상 불성실공시법인으로 지정', '화의 불인가', '회생절차폐지결정사유 및 당해 코스닥시장상장법인의 계속기업 가능성 유무를 고려한 결과 상장폐지기준에 해당한다고 결정'])

In [ ]:
## dict로 replace
panel['unlist_reason_ctg'] = panel['unlist_reason']

# values에 해당되면 key로 치환
key_list = []

for reason in panel['unlist_reason_ctg']:
  key = [k for k, v in ctg_dict.items() if reason in v]
  if len(key)==0:
    key_list.extend(reason)
  else:
    key_list.extend(key)

panel['unlist_reason_ctg'] = key_list
panel[panel['unlist_reason_ctg']!=panel['unlist_reason']][['unlist_reason_ctg', 'unlist_reason']]

,unlist_reason_ctg,unlist_reason
76864,자본전액잠식,자본전액잠식 2년 계속
76865,자본전액잠식,자본전액잠식 2년 계속
76866,자본전액잠식,자본전액잠식 2년 계속
76867,자본전액잠식,자본전액잠식 2년 계속
76868,자본전액잠식,자본전액잠식 2년 계속
...,...,...
111899,신청에 의한 상장폐지,상장폐지 신청('22.04.14)
111900,신청에 의한 상장폐지,상장폐지 신청('22.04.14)
111901,신청에 의한 상장폐지,상장폐지 신청('22.04.14)
111902,신청에 의한 상장폐지,상장폐지 신청('22.04.14)


## (3) 재무부실 사유 상장폐지기업 class 파생변수

In [ ]:
## 재무적 이유로 상장폐지 하지 않은 사유 필터
panel['fnc_rsn_unlst'] = panel['unlist_reason_ctg']

panel['fnc_rsn_unlst'] = panel['fnc_rsn_unlst'].replace(['해산 사유 발생', '영업활동정지 6월 계속', '자본전액잠식', '회사정리절차 폐지 결정', 
                                                         '감사의견거절', '주식양도 제한', '상장폐지유예기간종료', '부도', 
                                                         '보고서 또는 서류 미제출', '자본잠식률 50% 이상', '시가총액 미달', '보통주 주가수준미달', 
                                                         '당좌거래정지', '액면가액일정비율 미달', '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당',
                                                         '화의 불인가', '불성실공시법인지정', '2분기 연속 거래실적부진사유 미해소',
                                                         '회생절차폐지결정사유 및 당해 코스닥시장상장법인의 계속기업 가능성 유무를 고려한 결과 상장폐지기준에 해당한다고 결정'],"1")
panel['fnc_rsn_unlst'] = panel['fnc_rsn_unlst'].replace(['피흡수합병','자진등록취소','신청에 의한 상장폐지', '지주회사(최대주주등)의 완전자회사화 등',
                                                          '코스닥시장 상장', '기타 등록취소', '주된영업의 양도','계약이전'],"0")
panel[['unlist_reason_ctg', 'fnc_rsn_unlst']] # {상장폐지X, 그외사유 상장폐지: 0}, {재무사유 상장폐지: 1}

,unlist_reason_ctg,fnc_rsn_unlst
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
111899,신청에 의한 상장폐지,0
111900,신청에 의한 상장폐지,0
111901,신청에 의한 상장폐지,0
111902,신청에 의한 상장폐지,0


## (4) 기업코드 레이블 인코딩

In [ ]:
## 기업코드 레이블 인코딩
panel['code_label'] = [int(re.sub("^A", "", row) )for row in panel['code']]

In [ ]:
## 되돌리기 위한 매칭 함수 {기업코드_int:기업코드}
def df_to_dict(df, keys_col, values_col):
      
  trans = df.loc[:,[keys_col, values_col]].transpose()
  trans.columns = trans.iloc[0]
  trans = trans.iloc[1].to_dict()

  for k in list(trans.keys()):
    if pd.isnull(trans[k])==True:
      del trans[k]

  return trans

# code_dict = df_to_dict(panel, 'code_label','code')
# print(code_dict)

## (5) 결측치 제거 
- 재무정보columns('cash/assets':'retained_earning')이 모두 Nan값인 경우만 row drop
- 열 하나라도 Nan값이 아니라면 drop하지 않음
- 주가정보 제외하고 재무정보가 없으면

In [ ]:
panel = panel.dropna(subset = panel.loc[:,'cash/assets':'net_income_grow'].columns,axis = 0, how='all').reset_index(drop=True)
panel.apply(lambda x: pd.isna(x).sum()) 

code                                     0
cor_name                                 0
Accounting_Closing                       0
year                                     0
cash/assets                              6
liabilities/assets                       5
liabilities-cash/assets                  5
sales/cur_liabilities                    6
cur_assets/cur_liabilities               5
retained_earnings/cur_liabilities        6
non_cur_lia/non_cur_lia+worth            6
interest/cash_flow                      13
interest/ebitda                         23
inven/sales                            158
sales/net_income                        13
gross_margin/sales                     191
sales/retained_earnings                 15
change_in_roa                         9303
change_in_AR                          2896
change_in_TP                          4317
assets                                   5
sales                                    0
roa                                   4691
sales_grow 

In [ ]:
panel.shape

(55904, 44)

## (6) 상장폐지 날짜 class 형태 파생변수

### a. 상장폐지날짜 정보 기준 (panel_a)

In [ ]:
## 상장폐지일이 지난 후에도 재무정보가 있는 행 drop
panel['unlist_year'] = panel['unlist_year'].fillna(10000).astype(int)
panel_a = panel[panel['year']<=panel['unlist_year']].reset_index(drop=True) # year > unlist_year인 행 제외

In [ ]:
## 상장폐지일이 지난 후에 회계결산한 재무정보 행 drop (월이 같아도 회계결산은 월말에 이뤄지므로 같거나 회계결산월이 더 크면 drop)
panel_a['unlist_mon'] = panel_a['unlist_date'].str.slice(start=5, stop=7)
panel_a['unlist_mon'] = panel_a['unlist_mon'].fillna(10000).astype(float)
panel_a = panel_a[(panel_a['year']!=panel_a['unlist_year'])|((panel_a['year']==panel_a['unlist_year'])&(panel_a['Accounting_Closing']<panel_a['unlist_mon']))].reset_index(drop=True)
print(54920-220)
print(panel_a.shape)

54700
(54728, 45)


In [ ]:
## [선택]
### (1) 오직 전년도 재무정보를 가지고 다음해 상장폐지를 예측할 것인가? -> 아래 코드 필요 O
### (2) 상장폐지 당해 연도 재무정보가 반영되어 상장폐지가 결정된 경우도 고려할 것인가? (상장폐지 당해연도 재무정보 사용할 것인가?) -> 아래코드 필요X
print(54920-220-10)
# panel_a = panel_a[panel_a['year']!=panel_a['unlist_year']].reset_index(drop=True)
panel_a

54690


,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,상장일자,unlist_date,unlist_reason,bsn_code,grngas,unlist_year,unlist_reason_ctg,fnc_rsn_unlst,code_label,unlist_mon
0,A005930,삼성전자,12.0,1990,0.026243,0.805812,0.779569,2.738511,0.948362,0.223232,...,1975-06-11,NaN,0,124-81-00998,NaN,10000,0,0,5930,10000.0
1,A005930,삼성전자,12.0,1991,0.011561,0.825602,0.814041,1.887630,0.740129,0.145565,...,1975-06-11,NaN,0,124-81-00998,NaN,10000,0,0,5930,10000.0
2,A005930,삼성전자,12.0,1992,0.026253,0.819760,0.793507,2.032037,0.708325,0.146367,...,1975-06-11,NaN,0,124-81-00998,NaN,10000,0,0,5930,10000.0
3,A005930,삼성전자,12.0,1993,0.033530,0.793795,0.760265,2.495139,0.815311,0.162677,...,1975-06-11,NaN,0,124-81-00998,NaN,10000,0,0,5930,10000.0
4,A005930,삼성전자,12.0,1994,0.022999,0.692778,0.669779,3.189266,1.128719,0.412265,...,1975-06-11,NaN,0,124-81-00998,NaN,10000,0,0,5930,10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54723,A950180,SNK,7.0,2017,0.611887,0.168936,-0.442952,4.380337,5.477438,2.218058,...,2019-05-07,2022-05-18,상장폐지 신청('22.04.14),000-81-20024,NaN,2022,신청에 의한 상장폐지,0,950180,5.0
54724,A950180,SNK,7.0,2018,0.527099,0.165288,-0.361812,4.341584,6.356767,3.357695,...,2019-05-07,2022-05-18,상장폐지 신청('22.04.14),000-81-20024,NaN,2022,신청에 의한 상장폐지,0,950180,5.0
54725,A950180,SNK,7.0,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,...,2019-05-07,2022-05-18,상장폐지 신청('22.04.14),000-81-20024,NaN,2022,신청에 의한 상장폐지,0,950180,5.0
54726,A950180,SNK,7.0,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,...,2019-05-07,2022-05-18,상장폐지 신청('22.04.14),000-81-20024,NaN,2022,신청에 의한 상장폐지,0,950180,5.0


In [ ]:
## 위의 조건 drop후, 마지막 연도 재무정보에 라벨링 == '1'
panel_a['unlist_last_year'] = '0' 
unlist_cor = list(panel_a[panel_a['unlist_date'].notnull()]['cor_name'].unique())
fin_year_list=[]

for cor in unlist_cor:
  fin_year = max(list(panel_a[panel_a['cor_name']==cor]['year']))
  fin_idx = panel_a[(panel_a['cor_name']==cor)&(panel_a['year']==fin_year)].index
  panel_a['unlist_last_year'][fin_idx] = '1'

## 재무부실사유 상장폐지 아닌 기업 == 0
panel_a['fnc_rsn_unlst_year'] = panel_a['fnc_rsn_unlst']
panel_a['fnc_rsn_unlst_year']  = np.where((panel_a['fnc_rsn_unlst']=="1")&(panel_a['unlist_last_year']=="1"), "1", "0")

panel_a[panel_a['unlist_date'].notnull()][['cor_name', 'year','fnc_rsn_unlst','unlist_last_year','fnc_rsn_unlst_year','unlist_date']].tail(10)

<ipython-input-15-5c4f38002cc2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel_a['unlist_last_year'][fin_idx] = '1'


,cor_name,year,fnc_rsn_unlst,unlist_last_year,fnc_rsn_unlst_year,unlist_date
54718,중국고섬,2008,1,0,0,2013-10-04
54719,중국고섬,2009,1,0,0,2013-10-04
54720,중국고섬,2010,1,0,0,2013-10-04
54721,중국고섬,2011,1,0,0,2013-10-04
54722,중국고섬,2012,1,1,1,2013-10-04
54723,SNK,2017,0,0,0,2022-05-18
54724,SNK,2018,0,0,0,2022-05-18
54725,SNK,2019,0,0,0,2022-05-18
54726,SNK,2020,0,0,0,2022-05-18
54727,SNK,2021,0,1,0,2022-05-18


In [ ]:
## 순서 변경
panel_a = panel_a[['code', 'cor_name', 'Accounting_Closing', 'year', 'cash/assets', 'liabilities/assets', 'liabilities-cash/assets',
       'sales/cur_liabilities', 'cur_assets/cur_liabilities','retained_earnings/cur_liabilities', 'non_cur_lia/non_cur_lia+worth',
       'interest/cash_flow', 'interest/ebitda', 'inven/sales','sales/net_income', 'gross_margin/sales', 'sales/retained_earnings',
       'change_in_roa', 'change_in_AR', 'change_in_TP', 'assets', 'sales','roa', 'sales_grow', 'net_income', 'net_income_grow','gross_profit', 'operating_profit', 'retained_earning', 'closing_price',
       'revised_price', 'market_capital', 'return','code_label','fnc_rsn_unlst_year','코드대분류', '최초상장일자', '상장일자',
       'unlist_date', 'unlist_reason', 'unlist_year', 'unlist_reason_ctg', 'fnc_rsn_unlst', 'unlist_mon', 'unlist_last_year']]

In [ ]:
## 필요X
def cor_shift(df, period):
  shift_result = pd.DataFrame()

  for cor in df['code_label'].unique():
    df_cor = df[df['code_label']==cor]
    df_cor['y_shift'] = df_cor['fnc_rsn_unlst_year'].shift(periods=period)
    shift_result = pd.concat([shift_result, df_cor], axis=0)

  return shift_result

# panel_shift = cor_shift(panel_a, 1)

# #2. 데이터 형태 & 범위

- (1) 무지성 모든연도 때려넣기 (원데이터)
- (2) row -> col (원데이터)
- (3) 변화율 (변환)
- (4) 지수함수화, 최소자승법 (변환)

In [ ]:
def var1(df):
  X1 = df.loc[:,'cash/assets':'retained_earning']
  X1['year'] = df['year']
  X1['code_label'] = df['code_label']

  y = df['fnc_rsn_unlst_year']

  return X1, y

In [ ]:
def xgb_smltn(X, y):

  X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=1)

  # 객체 생성
  model = XGBClassifier(n_estimators=500, early_stopping_rounds=1, max_depth=3, random_state=0)
  xgb_model = model.fit(X_train, y_train, early_stopping_rounds=100,eval_set=[(X_test, y_test)])

  # 예측하기
  y_pre = xgb_model.predict(X_test)
  y_pred_probs = xgb_model.predict_proba(X_test)[:,1]

  # # 특성 중요도 시각화
  # fig, ax = plt.subplots(figsize=(10,12))
  # plot_importance(xgb_model, ax=ax)

  X_test['y_real'] = y_test
  X_test['y_pred'] = y_pre

  print("정확도:",accuracy_score(y_pre, y_test))

  return X_test

## (1) 원데이터 사용해 최적범위 탐색: 마지막 회계결산 재무정보로만 예측 
- 각 기업별 row는 마지막 재무정보로만 train 
- '데이터 범위(2)'에서 range(0)인 경우와 같음

In [ ]:
def train_last_fnc(df):
  train = pd.DataFrame()

  for cor in tqdm(df['code_label'].unique()):
    df_cor = df[df['code_label']==cor]

    fin_year = max(list(df_cor['year']))
    fin_idx = df_cor[df_cor['year']==fin_year]
    train = train.append(fin_idx)

  return train.reset_index(drop=True)

last_fnc_train = train_last_fnc(panel_a)

100%|██████████| 3047/3047 [00:15<00:00, 191.26it/s]


In [ ]:
# last_fnc_train = train_last_fnc(panel_a)

# X2_1, y2_1 = var1(last_fnc_train)
# last_fnc_result1 = xgb_smltn(X2_1, y2_1)

In [ ]:
# print("정확도:",accuracy_score(last_fnc_result1['y_pred'], last_fnc_result1['y_real']))

In [ ]:
# result_count = pd.DataFrame(last_fnc_result1.groupby(['y_real','y_pred'])['y_pred'].count()).rename(columns={'y_pred':'count'}).reset_index()
# print(last_fnc_result1.groupby(['y_real','y_pred'])['y_pred'].count(), '\n')
# print('상장폐지를 상장유지로 예측:', result_count['count'][1], '/',result_count['count'][0]+result_count['count'][1])
# print('상장유지를 상장폐지로 예측:',result_count['count'][2], '/',result_count['count'][2]+result_count['count'][3])
# print(round(result_count['count'][1]/(result_count['count'][0]+result_count['count'][1]),4))
# print(round(result_count['count'][2]/(result_count['count'][2]+result_count['count'][3]),4))

## (2) 원데이터: 특정 기간 연도의 재무정보를 하나의 row로 나열

In [ ]:
def train_row_to_col(df, contained_year):
  
  ## X
  X = pd.DataFrame()

  for cor in tqdm(df['code_label'].unique()):
    df_cor = df[df['code_label']==cor]

    if len(df_cor) >= contained_year:
      fin_year = max(list(df_cor['year']))
      fin_idx = df_cor[df_cor['year']==fin_year].index[0]
      df_cor = df_cor.loc[fin_idx-contained_year+1:fin_idx, 'cash/assets':'retained_earning']

      cor_result = pd.DataFrame()

      for row in df_cor.index:
        row_df = df_cor.loc[[row]].reset_index(drop=True)
        cor_result = pd.concat([cor_result, row_df], axis=1) #, names=[df_cor.columns for num in range(len(df_cor))])

      cor_result.columns = [col+'_'+str(num) for num in range(len(df_cor)) for col in df_cor.columns]
      cor_result['code_label'] = cor
      X = pd.concat([X, cor_result], ignore_index=True)
    else:
      pass

  ## y  
  y_list = []
  
  for row in tqdm(range(len(X))):
    row_cor_code = X['code_label'][row]
    fnc_unlst_cor = df[(df['code_label']==row_cor_code)&(df['fnc_rsn_unlst_year']=='1')]
    if len(fnc_unlst_cor)==1:
      y_list.append('1')
    elif len(fnc_unlst_cor)==0:
      y_list.append('0')
    else:
      print('error')
    
  X['fnc_rsn_unlst_year'] = y_list

  return X

In [ ]:
# train_data1 = train_row_to_col(panel_a, 3) #1년전 자료까지

# X1_2 = train_data1.iloc[:,:-1]
# y1_2 = train_data1['fnc_rsn_unlst_year']

# result1_2 = xgb_smltn(X1_2, y1_2)

In [ ]:
# print("정확도:",accuracy_score(result1_2['y_pred'], result1_2['y_real']))

In [ ]:
# result_count = pd.DataFrame(result1_2.groupby(['y_real','y_pred'])['y_pred'].count()).rename(columns={'y_pred':'count'}).reset_index()
# print(result1_2.groupby(['y_real','y_pred'])['y_pred'].count(), '\n')
# print('상장폐지를 상장유지로 예측:', result_count['count'][1], '/',result_count['count'][0]+result_count['count'][1])
# print('상장유지를 상장폐지로 예측:',result_count['count'][2], '/',result_count['count'][2]+result_count['count'][3])
# print(round(result_count['count'][1]/(result_count['count'][0]+result_count['count'][1]),4))
# print(round(result_count['count'][2]/(result_count['count'][2]+result_count['count'][3]),4))

## (3) 데이터 변환: 변화율, 증가율

In [ ]:
def growth_rate(p0, p1):
  rate = (p1 - p0)/p0 #증가율 계산 = (현재시점-이전시점)/(이전시점)
  return rate

In [ ]:
def train_growth_rate(df, contained_year): #마지막연도 재무정보 기준 n년 전
  
  ## X
  X = pd.DataFrame()

  for col in tqdm(df.columns[:-1]):
    rate_list = []
    cor_list = []

    for cor in df['code_label'].unique():
      df_cor = df[df['code_label']==cor][col].values

      if contained_year == 'all':
        p0 = df_cor[0]
        p1 = df_cor[-1]
      elif len(df_cor) > contained_year:
        p0 = df_cor[int(contained_year)-1]
        p1 = df_cor[-1]

      rate = growth_rate(p0, p1)

      cor_list.append(cor)
      rate_list.append(rate)

    X[col+'_'+'growth'] = rate_list

  X['code_label'] = cor_list
  X = X.drop(['code_label_growth'], axis=1)


  ## y  
  y_list = []
  
  for row in tqdm(range(len(X))):
    row_cor_code = X['code_label'][row]
    fnc_unlst_cor = df[(df['code_label']==row_cor_code)&(df['fnc_rsn_unlst_year']=='1')]
    if len(fnc_unlst_cor)==1:
      y_list.append('1')
    elif len(fnc_unlst_cor)==0:
      y_list.append('0')
    else:
      print('error')
    
  X['fnc_rsn_unlst_year'] = y_list

  return X

In [ ]:
# train_data1_3 = train_growth_rate(panel_a.loc[:,'cash/assets':'fnc_rsn_unlst_year'], 'all')  

# X1_3 = train_data1_3.iloc[:,:-1]
# y1_3 = train_data1_3['fnc_rsn_unlst_year']

# result1_3 = xgb_smltn(X1_3, y1_3)

In [ ]:
# print("정확도:",accuracy_score(result1_3['y_pred'], result1_3['y_real']))

In [ ]:
# result_count = pd.DataFrame(result1_3.groupby(['y_real','y_pred'])['y_pred'].count()).rename(columns={'y_pred':'count'}).reset_index()
# print(result1_3.groupby(['y_real','y_pred'])['y_pred'].count(), '\n')
# print('상장폐지를 상장유지로 예측:', result_count['count'][1], '/',result_count['count'][0]+result_count['count'][1])
# print('상장유지를 상장폐지로 예측:',result_count['count'][2], '/',result_count['count'][2]+result_count['count'][3])
# print(round(result_count['count'][1]/(result_count['count'][0]+result_count['count'][1]),4))
# print(round(result_count['count'][2]/(result_count['count'][2]+result_count['count'][3]),4))

## (4) 데이터 변환: 최소제곱법 기반 지수함수 기울기

In [ ]:
def non_linear_least_square(x,Y):
    b = (np.mean(x)*np.mean(Y) - np.mean(x*Y))/(np.mean(x)*np.mean(x)-np.mean(x*x))
    A = np.mean(Y)-np.mean(x)*b
    a = np.exp(A)
    return a,b

In [ ]:
def train_non_linear(df, contained_year):
  
  ## X
  X = pd.DataFrame()

  for col in tqdm(df.columns[:-1]):
    a_list = []
    b_list = []
    cor_list = []

    for cor in df['code_label'].unique():
      df_cor = df[df['code_label']==cor][col]

      if contained_year == 'all':
        y = df[df['code_label']==cor][col].values
      elif len(df_cor) >= contained_year:
        y = df_cor.iloc[len(df_cor)-contained_year:len(df_cor)].values

      x = np.array([inx for inx in range(len(y))])
      Y = np.log(y)

      a,b = non_linear_least_square(x,Y)

      cor_list.append(cor)
      a_list.append(a)
      b_list.append(b)

    X[col+'_'+'a'] = a_list
    X[col+'_'+'b'] = b_list

  X['code_label'] = cor_list
  X = X.drop(['code_label_a',	'code_label_b'], axis=1)


  ## y  
  y_list = []
  
  for row in tqdm(range(len(X))):
    row_cor_code = X['code_label'][row]
    fnc_unlst_cor = df[(df['code_label']==row_cor_code)&(df['fnc_rsn_unlst_year']=='1')]
    if len(fnc_unlst_cor)==1:
      y_list.append('1')
    elif len(fnc_unlst_cor)==0:
      y_list.append('0')
    else:
      print('error')
    
  X['fnc_rsn_unlst_year'] = y_list

  return X

In [ ]:
# train_data1_4 = train_non_linear(panel_a.loc[:,'cash/assets':'fnc_rsn_unlst_year'], 'all')  # 각 기업별 전범위는 contained_year='all'

# X1_4 = train_data1_4.iloc[:,:-1]
# y1_4 = train_data1_4['fnc_rsn_unlst_year']

# result1_4 = xgb_smltn(X1_4, y1_4)

In [ ]:
# print("정확도:",accuracy_score(result1_4['y_pred'], result1_4['y_real']))

In [ ]:
# result_count = pd.DataFrame(result1_4.groupby(['y_real','y_pred'])['y_pred'].count()).rename(columns={'y_pred':'count'}).reset_index()
# print(result1_4.groupby(['y_real','y_pred'])['y_pred'].count(), '\n')
# print('상장폐지를 상장유지로 예측:', result_count['count'][1], '/',result_count['count'][0]+result_count['count'][1])
# print('상장유지를 상장폐지로 예측:',result_count['count'][2], '/',result_count['count'][2]+result_count['count'][3])
# print(round(result_count['count'][1]/(result_count['count'][0]+result_count['count'][1]),4))
# print(round(result_count['count'][2]/(result_count['count'][2]+result_count['count'][3]),4))

# #4. 최적 데이터 형태 및 범위 조합한 모델

## (1) 마지막 회계결산 재무정보로만 예측 + 증가율(n=4)

In [ ]:
def train_last_grwth(df, contained_year): #주식 포함

  last_fnc_train = train_last_fnc(df).drop(['fnc_rsn_unlst_year', '코드대분류','최초상장일자', '상장일자', 'unlist_date', 'unlist_reason', 'unlist_year','unlist_reason_ctg', 'fnc_rsn_unlst', 'unlist_mon', 'unlist_last_year'], axis=1)
  train_growth_data = train_growth_rate(df.loc[:,'cash/assets':'fnc_rsn_unlst_year'], contained_year)

  result = last_fnc_train.merge(train_growth_data, how='outer',on='code_label')

  return result
  
union_train1 = train_last_grwth(panel_a, 4)

 39%|███▉      | 1186/3047 [00:05<00:08, 229.89it/s]


KeyboardInterrupt: ignored

In [ ]:
X4_1 = union_train1.loc[:,'year':'return_growth']
y4_1 = union_train1['fnc_rsn_unlst_year']
union_result4_1 = xgb_smltn(X4_1, y4_1)

In [ ]:
X4_1.columns

In [ ]:
print("정확도:",accuracy_score(union_result4_1['y_pred'], union_result4_1['y_real']))

In [ ]:
result_count = pd.DataFrame(union_result4_1.groupby(['y_real','y_pred'])['y_pred'].count()).rename(columns={'y_pred':'count'}).reset_index()
print(union_result4_1.groupby(['y_real','y_pred'])['y_pred'].count(), '\n')
print('상장폐지를 상장유지로 예측:', result_count['count'][1], '/',result_count['count'][0]+result_count['count'][1])
print('상장유지를 상장폐지로 예측:',result_count['count'][2], '/',result_count['count'][2]+result_count['count'][3])
print(round(result_count['count'][1]/(result_count['count'][0]+result_count['count'][1]),4))
print(round(result_count['count'][2]/(result_count['count'][2]+result_count['count'][3]),4))

## (2) 마지막 회계결산 재무정보로만 예측 + 지수함수(n=2)

In [ ]:
def train_last_non_lnr(df, contained_year):

  last_fnc_train = train_last_fnc(df).drop(['fnc_rsn_unlst_year', '코드대분류','최초상장일자', '상장일자', 'unlist_date', 'unlist_reason', 'unlist_year','unlist_reason_ctg', 'fnc_rsn_unlst', 'unlist_mon', 'unlist_last_year'], axis=1)
  train_non_linea_data = train_non_linear(df.loc[:,'cash/assets':'fnc_rsn_unlst_year'], contained_year)

  result = last_fnc_train.merge(train_non_linea_data, how='outer',on='code_label')

  return result
  
union_train2 = train_last_non_lnr(panel_a, 2)

  0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-34-649cbbb9e786>:20: RuntimeWarning: divide by zero encountered in log
  Y = np.log(y)
<ipython-input-33-77dc735660f7>:2: RuntimeWarning: invalid value encountered in double_scalars
  b = (np.mean(x)*np.mean(Y) - np.mean(x*Y))/(np.mean(x)*np.mean(x)-np.mean(x*x))
<ipython-input-33-77dc735660f7>:2: RuntimeWarning: invalid value encountered in multiply
  b = (np.mean(x)*np.mean(Y) - np.mean(x*Y))/(np.mean(x)*np.mean(x)-np.mean(x*x))
<ipython-input-34-649cbbb9e786>:20: RuntimeWarning: invalid value encountered in log
  Y = np.log(y)
100%|██████████| 3047/3047 [00:18<00:00, 168.61it/s]


In [ ]:
X4_2 = union_train2.loc[:,'year':'return_b']
y4_2 = union_train2['fnc_rsn_unlst_year']
union_result4_2 = xgb_smltn(X4_2, y4_2)

In [ ]:
X4_2.columns

Index(['year', 'cash/assets', 'liabilities/assets', 'liabilities-cash/assets',
       'sales/cur_liabilities', 'cur_assets/cur_liabilities',
       'retained_earnings/cur_liabilities', 'non_cur_lia/non_cur_lia+worth',
       'interest/cash_flow', 'interest/ebitda', 'inven/sales',
       'sales/net_income', 'gross_margin/sales', 'sales/retained_earnings',
       'change_in_roa', 'change_in_AR', 'change_in_TP', 'assets', 'sales',
       'roa', 'sales_grow', 'net_income', 'net_income_grow', 'gross_profit',
       'operating_profit', 'retained_earning', 'closing_price',
       'revised_price', 'market_capital', 'return', 'cash/assets_a',
       'cash/assets_b', 'liabilities/assets_a', 'liabilities/assets_b',
       'liabilities-cash/assets_a', 'liabilities-cash/assets_b',
       'sales/cur_liabilities_a', 'sales/cur_liabilities_b',
       'cur_assets/cur_liabilities_a', 'cur_assets/cur_liabilities_b',
       'retained_earnings/cur_liabilities_a',
       'retained_earnings/cur_liabilities_b

In [ ]:
print("정확도:",accuracy_score(union_result4_2['y_pred'], union_result4_2['y_real']))

정확도: 0.9278688524590164


In [ ]:
result_count = pd.DataFrame(union_result4_2.groupby(['y_real','y_pred'])['y_pred'].count()).rename(columns={'y_pred':'count'}).reset_index()
print(union_result4_2.groupby(['y_real','y_pred'])['y_pred'].count(), '\n')
print('상장폐지를 상장유지로 예측:', result_count['count'][1], '/',result_count['count'][0]+result_count['count'][1])
print('상장유지를 상장폐지로 예측:',result_count['count'][2], '/',result_count['count'][2]+result_count['count'][3])
print(round(result_count['count'][1]/(result_count['count'][0]+result_count['count'][1]),4))
print(round(result_count['count'][2]/(result_count['count'][2]+result_count['count'][3]),4))

y_real  y_pred
0       0         466
        1          38
1       0           6
        1         100
Name: y_pred, dtype: int64 

상장폐지를 상장유지로 예측: 38 / 504
상장유지를 상장폐지로 예측: 6 / 106
0.0754
0.0566


## (3) 마지막 회계결산 재무정보로만 예측 + 증가율(n=4) + 지수함수(n=2)

In [ ]:
def train_last_grwth_non_lnr(df, contained_year1, contained_year2):

  last_fnc_train = train_last_fnc(df).drop(['fnc_rsn_unlst_year', '코드대분류','최초상장일자', '상장일자', 'unlist_date', 'unlist_reason', 'unlist_year','unlist_reason_ctg', 'fnc_rsn_unlst', 'unlist_mon', 'unlist_last_year'], axis=1)
  train_growth_data = train_growth_rate(df.loc[:,'cash/assets':'fnc_rsn_unlst_year'], contained_year1)
  train_non_linea_data = train_non_linear(df.loc[:,'cash/assets':'fnc_rsn_unlst_year'], contained_year2)

  result = last_fnc_train.merge(train_growth_data.drop(['fnc_rsn_unlst_year'],axis=1), how='outer',on='code_label')
  result = result.merge(train_non_linea_data, how='outer',on='code_label')

  return result
  
union_train3 = train_last_grwth_non_lnr(panel_a, 4, 2)

  0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-28-70b133b319d3>:2: RuntimeWarning: divide by zero encountered in double_scalars
  rate = (p1 - p0)/p0 #증가율 계산 = (현재시점-이전시점)/(이전시점)
<ipython-input-28-70b133b319d3>:2: RuntimeWarning: invalid value encountered in double_scalars
  rate = (p1 - p0)/p0 #증가율 계산 = (현재시점-이전시점)/(이전시점)
  0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-34-649cbbb9e786>:20: RuntimeWarning: divide by zero encountered in log
  Y = np.log(y)
<ipython-input-33-77dc735660f7>:2: RuntimeWarning: invalid value encountered in multiply
  b = (np.mean(x)*np.mean(Y) - np.mean(x*Y))/(np.mean(x)*np.mean(x)-np.mean(x*x))
<ipython-input-34-649cbbb9e786>:20: RuntimeWarning: invalid value encountered in log
  Y = np.log(y)
100%|██████████| 3047/3047 [00:18<00:00, 161.50it/s]


In [ ]:
union_X3 = union_train3.loc[:,'year':'return_b']
union_y3 = union_train3['fnc_rsn_unlst_year']

union_result3 = xgb_smltn(union_X3, union_y3)

In [ ]:
union_X3.columns

Index(['year', 'cash/assets', 'liabilities/assets', 'liabilities-cash/assets',
       'sales/cur_liabilities', 'cur_assets/cur_liabilities',
       'retained_earnings/cur_liabilities', 'non_cur_lia/non_cur_lia+worth',
       'interest/cash_flow', 'interest/ebitda',
       ...
       'retained_earning_a', 'retained_earning_b', 'closing_price_a',
       'closing_price_b', 'revised_price_a', 'revised_price_b',
       'market_capital_a', 'market_capital_b', 'return_a', 'return_b'],
      dtype='object', length=118)

In [ ]:
print("정확도:",accuracy_score(union_result3['y_pred'], union_result3['y_real']))

정확도: 0.9295081967213115


In [ ]:
result_count = pd.DataFrame(union_result3.groupby(['y_real','y_pred'])['y_pred'].count()).rename(columns={'y_pred':'count'}).reset_index()
print(union_result3.groupby(['y_real','y_pred'])['y_pred'].count(), '\n')
print('상장폐지를 상장유지로 예측:', result_count['count'][1], '/',result_count['count'][0]+result_count['count'][1])
print('상장유지를 상장폐지로 예측:',result_count['count'][2], '/',result_count['count'][2]+result_count['count'][3])
print(round(result_count['count'][1]/(result_count['count'][0]+result_count['count'][1]),4))
print(round(result_count['count'][2]/(result_count['count'][2]+result_count['count'][3]),4))

y_real  y_pred
0       0         465
        1          39
1       0           4
        1         102
Name: y_pred, dtype: int64 

상장폐지를 상장유지로 예측: 39 / 504
상장유지를 상장폐지로 예측: 4 / 106
0.0774
0.0377


## (4) 마지막 회계결산 재무정보로만 예측 + row로 나열(n=2) + 증가율(n=4) + 지수함수(n=2)

In [ ]:
def train_last_row_grwth_non_lnr(df, contained_year1, contained_year2, contained_year3):

  last_fnc_data = train_last_fnc(df).drop(['fnc_rsn_unlst_year', '코드대분류','최초상장일자', '상장일자', 'unlist_date', 'unlist_reason', 'unlist_year','unlist_reason_ctg', 'fnc_rsn_unlst', 'unlist_mon', 'unlist_last_year'], axis=1)
  train_row_data = train_row_to_col(panel_a.loc[:,'year':'fnc_rsn_unlst_year'], contained_year2).drop(['fnc_rsn_unlst_year'], axis=1)
  train_growth_data = train_growth_rate(panel_a.loc[:,'cash/assets':'fnc_rsn_unlst_year'], contained_year2).drop(['fnc_rsn_unlst_year'], axis=1)
  train_non_linea_data = train_non_linear(panel_a.loc[:,'cash/assets':'fnc_rsn_unlst_year'], contained_year3)

  result = last_fnc_data.merge(train_row_data, how='outer',on='code_label')
  result = result.merge(train_growth_data, how='outer',on='code_label')
  result = result.merge(train_non_linea_data, how='outer',on='code_label')

  return result
  
union_train4 = train_last_row_grwth_non_lnr(panel_a, 2, 4, 2)

 18%|█▊        | 544/3047 [00:01<00:08, 289.26it/s]


KeyboardInterrupt: ignored

In [ ]:
union_train4

In [ ]:
union_X4 = union_train4.loc[:,'year':'return_b']
union_y4 = union_train4['fnc_rsn_unlst_year']

union_result4 = xgb_smltn(union_X4, union_y4)

In [ ]:
union_X4.columns

In [ ]:
print("정확도:",accuracy_score(union_result4['y_pred'], union_result4['y_real']))

In [ ]:
result_count = pd.DataFrame(union_result4.groupby(['y_real','y_pred'])['y_pred'].count()).rename(columns={'y_pred':'count'}).reset_index()
print(union_result4.groupby(['y_real','y_pred'])['y_pred'].count(), '\n')
print('상장폐지를 상장유지로 예측:', result_count['count'][1], '/',result_count['count'][0]+result_count['count'][1])
print('상장유지를 상장폐지로 예측:',result_count['count'][2], '/',result_count['count'][2]+result_count['count'][3])
print(round(result_count['count'][1]/(result_count['count'][0]+result_count['count'][1]),4))
print(round(result_count['count'][2]/(result_count['count'][2]+result_count['count'][3]),4))

# #5. 응용 모델 & 주식데이터 유무

## (1)

In [ ]:
def without_stock(df):
  X = df.loc[:,'year':'retained_earning']
  X['code_label'] = df['code_label']
  X['fnc_rsn_unlst_year'] = df['fnc_rsn_unlst_year']

  return X

panel_a_without_stock = without_stock(panel_a)

In [ ]:
def train_last_row_grwth_non_lnr(df, contained_year1, contained_year2, contained_year3):

  df_without_stock = without_stock(df)#.drop(['year'], axis=1)

  last_fnc_data = train_last_fnc(df).drop(['fnc_rsn_unlst_year', '코드대분류','최초상장일자', '상장일자', 'unlist_date', 'unlist_reason', 'unlist_year','unlist_reason_ctg', 'fnc_rsn_unlst', 'unlist_mon', 'unlist_last_year'], axis=1)
  train_row_data = train_row_to_col(df_without_stock.loc[:,'year':'fnc_rsn_unlst_year'], contained_year2).drop(['fnc_rsn_unlst_year'], axis=1)
  train_growth_data = train_growth_rate(df.loc[:,'cash/assets':'fnc_rsn_unlst_year'], contained_year2).drop(['fnc_rsn_unlst_year'], axis=1)
  train_non_linea_data = train_non_linear(df.loc[:,'cash/assets':'fnc_rsn_unlst_year'], contained_year3)

  result = last_fnc_data.merge(train_row_data, how='outer',on='code_label')
  result = result.merge(train_growth_data, how='outer',on='code_label')
  result = result.merge(train_non_linea_data, how='outer',on='code_label')

  return result
  
union_train5 = train_last_row_grwth_non_lnr(panel_a, 2, 4, 2)

  0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-28-70b133b319d3>:2: RuntimeWarning: divide by zero encountered in double_scalars
  rate = (p1 - p0)/p0 #증가율 계산 = (현재시점-이전시점)/(이전시점)
<ipython-input-28-70b133b319d3>:2: RuntimeWarning: invalid value encountered in double_scalars
  rate = (p1 - p0)/p0 #증가율 계산 = (현재시점-이전시점)/(이전시점)
  0%|          | 0/30 [00:00<?, ?it/s]<ipython-input-34-649cbbb9e786>:20: RuntimeWarning: divide by zero encountered in log
  Y = np.log(y)
<ipython-input-33-77dc735660f7>:2: RuntimeWarning: invalid value encountered in multiply
  b = (np.mean(x)*np.mean(Y) - np.mean(x*Y))/(np.mean(x)*np.mean(x)-np.mean(x*x))
<ipython-input-34-649cbbb9e786>:20: RuntimeWarning: invalid value encountered in log
  Y = np.log(y)
100%|██████████| 3047/3047 [00:16<00:00, 187.92it/s]


In [ ]:
union_train5

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,retained_earning_b,closing_price_a,closing_price_b,revised_price_a,revised_price_b,market_capital_a,market_capital_b,return_a,return_b,fnc_rsn_unlst_year
0,A005930,삼성전자,12.0,2021,0.091490,0.285315,0.193825,3.173104,2.475832,3.325854,...,0.078023,81000.0,-0.033902,55200.0,0.407878,5.441168e+08,-0.033818,NaN,NaN,0
1,A373220,LG에너지솔루션,12.0,2021,0.053984,0.632119,0.578135,1.884302,1.006524,0.035633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,A000660,SK하이닉스,12.0,2021,0.052476,0.354774,0.302298,2.911284,1.819338,3.777014,...,0.171432,118500.0,0.100284,94700.0,0.285568,8.626828e+07,0.100284,25.93,-0.899275,0
3,A207940,삼성바이오로직스,12.0,2021,0.005949,0.373765,0.367816,1.416070,2.549615,2.119749,...,0.183545,826000.0,0.089128,423485.0,0.659930,5.465229e+07,0.089128,90.76,-2.276056,0
4,A006400,삼성SDI,12.0,2021,0.090027,0.411737,0.321710,2.097604,1.152233,1.318077,...,0.138079,628000.0,0.042095,232000.0,1.062032,4.367758e+07,0.043103,166.10,-3.653975,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042,A900180,완리,12.0,2017,0.170397,0.315057,0.144660,1.837878,1.693219,2.568629,...,0.024877,1210.0,-0.971506,2007.0,-0.660309,1.240340e+05,-0.922733,NaN,NaN,1
3043,A950010,평산차업 KDR,12.0,2014,0.296593,0.284505,-0.012087,0.525747,2.115551,-0.425187,...,NaN,1525.0,-0.793058,1510.0,-0.075637,7.325000e+03,-0.793137,NaN,NaN,1
3044,A950030,네프로아이티,3.0,2010,0.073408,0.707520,0.634112,0.880812,0.828345,-1.541515,...,NaN,5010.0,-1.222778,8811.0,-0.425063,2.505000e+04,-1.040456,NaN,NaN,1
3045,A950070,중국고섬,12.0,2012,0.008260,0.425390,0.417130,0.599048,0.343097,-0.671742,...,NaN,4165.0,-0.000000,5900.0,-0.348236,1.249500e+05,-0.000000,NaN,NaN,1


In [ ]:
union_X5 = union_train5.loc[:,'year':'retained_earning_b']
union_y5 = union_train5['fnc_rsn_unlst_year']

union_result5 = xgb_smltn(union_X5, union_y5)

[0]	validation_0-error:0.078689
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.077049
[2]	validation_0-error:0.077049
[3]	validation_0-error:0.080328
[4]	validation_0-error:0.07541
[5]	validation_0-error:0.07541
[6]	validation_0-error:0.07541
[7]	validation_0-error:0.07541
[8]	validation_0-error:0.072131
[9]	validation_0-error:0.072131
[10]	validation_0-error:0.07377
[11]	validation_0-error:0.07377
[12]	validation_0-error:0.072131
[13]	validation_0-error:0.072131
[14]	validation_0-error:0.072131
[15]	validation_0-error:0.072131
[16]	validation_0-error:0.072131
[17]	validation_0-error:0.072131
[18]	validation_0-error:0.072131
[19]	validation_0-error:0.072131
[20]	validation_0-error:0.072131
[21]	validation_0-error:0.072131
[22]	validation_0-error:0.072131
[23]	validation_0-error:0.072131
[24]	validation_0-error:0.070492
[25]	validation_0-error:0.070492
[26]	validation_0-error:0.07377
[27]	validation_0-error:0.072131
[28]	validation_0-error:0.

In [ ]:
print("정확도:",accuracy_score(union_result5['y_pred'], union_result5['y_real']))

정확도: 0.9295081967213115


In [ ]:
result_count = pd.DataFrame(union_result5.groupby(['y_real','y_pred'])['y_pred'].count()).rename(columns={'y_pred':'count'}).reset_index()
print(union_result5.groupby(['y_real','y_pred'])['y_pred'].count(), '\n')
print('상장폐지를 상장유지로 예측:', result_count['count'][1], '/',result_count['count'][0]+result_count['count'][1])
print('상장유지를 상장폐지로 예측:',result_count['count'][2], '/',result_count['count'][2]+result_count['count'][3])
print(round(result_count['count'][1]/(result_count['count'][0]+result_count['count'][1]),4))
print(round(result_count['count'][2]/(result_count['count'][2]+result_count['count'][3]),4))

y_real  y_pred
0       0         465
        1          39
1       0           4
        1         102
Name: y_pred, dtype: int64 

상장폐지를 상장유지로 예측: 39 / 504
상장유지를 상장폐지로 예측: 4 / 106
0.0774
0.0377


# 5. Bayesian Optimization

In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
union_X5 = union_train5.loc[:,'year':'retained_earning_b']
union_y5 = union_train5['fnc_rsn_unlst_year']

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb

# MAPE Metric
def mean_absolute_percentage_error(y_test, y_pred):
  y_test = np.array(y_test, dtype=np.int64)
  y_pred = np.array(y_pred, dtype=np.int64)
  return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

# 탐색 대상 함수 (XGBRegressor)
def XGB_cv(max_depth,learning_rate, n_estimators, gamma, min_child_weight, subsample, colsample_bytree, silent=True, nthread=-1):

  X_train,X_test,y_train,y_test= train_test_split(union_X5, union_y5,test_size=0.2,random_state=1)

  # 모델 정의
  model = xgb.XGBClassifier(max_depth=int(max_depth),
                           learning_rate=learning_rate,
                           n_estimators=int(n_estimators),
                           gamma=gamma,
                           min_child_weight=min_child_weight,
                           subsample=subsample,
                           colsample_bytree=colsample_bytree, 
                           nthread=nthread
                           )
  # 모델 훈련
  model.fit(X_train, y_train)

  # 예측값 출력
  y_pred= model.predict(X_test)

  # 각종 metric 계산
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  r2 = r2_score(y_test, y_pred)
  mape = mean_absolute_percentage_error(y_test, y_pred)

  # 오차 최적화로 사용할 metric 반환
  return r2

In [ ]:
#  bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
from bayes_opt import BayesianOptimization
import numpy as np

# 실험해보고자하는 hyperparameter 집합
pbounds = {'max_depth': (3, 7), 
           'learning_rate': (0.01, 0.2),
           'n_estimators': (5000, 10000),
           'gamma': (0, 100),
           'min_child_weight': (0, 3),
           'subsample': (0.5, 1),
           'colsample_bytree' :(0.2, 1)
           }

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
bo = BayesianOptimization(f=XGB_cv, pbounds=pbounds, verbose=2, random_state=1 )    

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)
bo.maximize(init_points=2, n_iter=10, acq='ei', xi=0.01)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(bo.max)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------


<ipython-input-136-ac2aecab6b5c>:26: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  bo.maximize(init_points=2, n_iter=10, acq='ei', xi=0.01)
<ipython-input-135-73a8a83e958e>:8: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
<ipython-input-135-73a8a83e958e>:8: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_test - y_pred) / y_test)) * 100


| 1         | 0.3378    | 0.5336    | 72.03     | 0.01002   | 4.209     | 0.4403    | 5.462e+03 | 0.5931    |


<ipython-input-135-73a8a83e958e>:8: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
<ipython-input-135-73a8a83e958e>:8: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_test - y_pred) / y_test)) * 100


| 2         | 0.3492    | 0.4764    | 39.68     | 0.1124    | 4.677     | 2.056     | 6.022e+03 | 0.9391    |


...

| 9         | 0.4177    | 0.9401    | 83.82     | 0.01144   | 6.577     | 0.0161    | 7.544e+03 | 0.7222    |


<ipython-input-135-73a8a83e958e>:8: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
<ipython-input-135-73a8a83e958e>:8: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_test - y_pred) / y_test)) * 100


| 10        | 0.5204    | 0.9855    | 19.81     | 0.1866    | 4.661     | 1.213     | 5.081e+03 | 0.8903    |


<ipython-input-135-73a8a83e958e>:8: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
<ipython-input-135-73a8a83e958e>:8: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_test - y_pred) / y_test)) * 100


| 11        | 0.2007    | 0.7031    | 82.86     | 0.1284    | 4.32      | 0.8823    | 5.455e+03 | 0.5068    |
